<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/report_metr_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import pandas as pd
import numpy as np
import re
import zipfile
from collections import namedtuple

In [ ]:
data = pd.read_excel('/content/Для анализа.xlsx', index_col='2')
data.head()

# 1 Обработка пропусков в данных

In [242]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

In [243]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

In [248]:
data['Есть повторно выданные замечания'] = np.where(data['Повторные замечания (шифры)'].isna(), 0, 1)
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)
data['Повторные замечания (шифры)'] = data['Повторные замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [249]:
# формируем список нерабочих дней за 2024 год согласно производственному календарю

# список всех СБ и ВС в 2024 году
sat_sun = pd.bdate_range(start='2024-01-01', end='2024-12-31', freq='C', weekmask='Sat Sun')
# рабочие СБ и ВС
work_weekends = {'2024-04-27', '2024-11-02', '2024-12-28'}
# праздничные дни
holidays = {
    '2024-01-01',
    '2024-01-02',
    '2024-01-03',
    '2024-01-04',
    '2024-01-05',
    '2024-01-08',
    '2024-02-23',
    '2024-03-08',
    '2024-04-29',
    '2024-04-30',
    '2024-05-01',
    '2024-05-09',
    '2024-05-10',
    '2024-06-12',
    '2024-11-04',
    '2024-12-30',
    '2024-12-31',
}
# формируем множество всех СБ и ВС в формате строки "2024-12-29"
not_workdays = set()

for d in sat_sun:
    not_workdays.add(str(d)[:10])

# формируем итоговое множество как объединение множества всех СБ и ВС
# с множество праздничных дней, а затем исключаем из него рабочие СБ и ВС,
# далее для удобства преобразуем в список
all_not_workdays = list(not_workdays.union(holidays) - work_weekends)

# ну и на всякий случай проверка, 366 дней в 2024 году, согласно производственному
# календарю рабочих дней 248, а значит нерабочих: 366 - 248 = 118 дней
assert(len(all_not_workdays)==118)

In [250]:
data['Срок проверки'] = [
    len(pd.bdate_range(
        start=data['Дата получения документа'].iloc[i],
        end=data['Дата возврата документа'].iloc[i],
        freq='C',
        weekmask='Mon Tue Wed Thu Fri Sat Sun',
        holidays=all_not_workdays,
        )) for i in range(data['Дата возврата документа'].shape[0])
    ]

In [251]:
data['Разработчик'] = ''

In [252]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
    data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
    for s in data['Отдел, разработчик'].iloc[i]:
        s = s.strip()
        data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
    data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-252-63d391b8659b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
<ipython-input-252-63d391b8659b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
<ipython-input-252-63d391b8659b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [253]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРСДВ', 'ОРС ДВ'], 'ОРС ДНГ и ВОГ')

data['Разработчик'] = data['Разработчик'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Мальцева', 'Мацюк'], 'Мацюк (Мальцева)')

In [ ]:
# добавляем к фамилии разработчика суффикс с названием его отдела
for i in range(data['Разработчик'].shape[0]):
    data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]

# 3 Обработка столбца с замечаниями

In [ ]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
    a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
    data[' Замечания (шифры)'].iloc[i] = a

for i in range(data['Повторные замечания (шифры)'].shape[0]):
    a = [s.strip() for s in re.split('[,.]', str(data['Повторные замечания (шифры)'].iloc[i]))]
    data['Повторные замечания (шифры)'].iloc[i] = a

In [256]:
data.sample(10)

,Дата получения документа,Дата возврата документа,Номер извещения,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры),Есть повторно выданные замечания,Срок проверки,Разработчик
2,,,,,,,,,,,,,,
548,2024-03-19,2024-03-19,NaN,ИТКС.75 2/2-24,ИТКС.687427.234 Плата управления,КО,5,Марусина,NaN,[0],[0],0,1,Крючкова_КО
1422,2024-06-11,2024-06-13,ИТКС.322-24,КМИВ.461534.027 ЭТ,КМИВ.461534.027 ЦПС-14,ОРЭ,4,Иванов,NaN,[0],[0],0,2,Казакова_ОРЭ
1166,2024-05-24,2024-05-28,NaN,ИТКС.469141.007 Д2,ИТКС.469141.007 МСП-7,ОРИС,7,Иванов,NaN,[0],[0],0,3,Храмкин_ОРИС
1006,2024-05-08,2024-05-08,ИТКС.316-24,ИТКС.712427.001,Заглушка,КО,2,Авилов,NaN,[0],[0],0,1,Миннеахметова_КО
363,2024-02-20,2024-02-20,NaN,ИТКС.685666.003 СБ,Жгут,ОРС ТВГ,4,Марусина,NaN,"[17, 18]",[0],0,1,Ахметзянов_ОРС ТВГ
1567,2024-06-20,2024-06-20,ИТКС.433 1/2-24,ИТКС.433 1/2-24,ИТКС.436221.003 ГН-1РС,ОРЭ,1,Иванов,присвоение литеры,[0],[0],0,1,Дорофеева_ОРЭ
494,2024-03-14,2024-03-14,NaN,ИТКС.745422.042,Прижим,КО,1,Марусина,NaN,[0],[0],0,1,Хохлов_КО
184,2024-01-29,2024-01-29,NaN,ИТКС.741134.124,Прокладка,КО,1,Марусина,NaN,[0],[0],0,1,Авилов_КО
242,2024-02-01,2024-02-01,NaN,ИТКС.402132.002 ТУ,ИТКС.402132.002-02 ПГИ-1Р-2,ОРС ДНГ и ВОГ,2,Иванов,NaN,[0],[0],0,1,Фролова_ОРС ДНГ и ВОГ


In [155]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):

        # если строка относится к нужному отделу
        if data['Отдел, разработчик'].iloc[i] == department:
            # берем значение из столбца с замечаниями

            for c in data[' Замечания (шифры)'].iloc[i]:
                print(i)
                # добавляем к счетчику шифров
                codes[int(c)] += 1

            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1

    return codes

In [257]:
def count_codes(data: pd.DataFrame, department: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    repeated_codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):
        # если строка относится к нужному отделу
        if data['Отдел, разработчик'].iloc[i] == department:
            # берем значение из столбца с замечаниями

            for c in data[' Замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров
                codes[int(c)] += 1
            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1
                    repeated_codes[int(c)] += 1

    return codes, repeated_codes

In [258]:
# функция для подсчета количества ошибок по разработчику

def count_codes_by_person(data: pd.DataFrame, person: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):

        # если строка относится к нужному разработчику
        if data['Разработчик'].iloc[i] == person:

            # берем значение из столбца с замечаниями
            for c in data[' Замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров
                codes[int(c)] += 1

            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1

    return codes

# 4 Формирование отчетов

In [272]:
def make_main_report(data: pd.DataFrame) -> pd.DataFrame:
    report = pd.DataFrame(columns=['Проверено документов, шт',
                                    'Проверено листов А4, шт',
                                    'Всего замечаний, шт',
                                    'Всего замечаний без 17 кода, шт',
                                    'Доля документов с замечаниями, %',
                                    'Количество замечаний на 100 листов А4, шт',
                                    'Количество замечаний на 100 листов А4 без 17 кода, шт',
                                    'Доля документов с повторно выданными замечаниями, %'])
    report['Проверено документов, шт'] = data.groupby(['Отдел, разработчик']).count()['Дата получения документа']
    report['Проверено листов А4, шт'] = data[['Отдел, разработчик', 'Количество форматов А4']].groupby(['Отдел, разработчик']).sum()

    # в цикле создаем вспомогательные переменные для подсчета замечаний и показателей
    for name in report.index.to_list():
        # создаем словарь для каждого отдела с количеством замечаний
        codes = count_codes(data, name)[0]
        # переменная для хранения количества замечаний без учета шифра '17'
        total_without_17 = 0
        # переменная для хранения количества замечаний с шифром '17'
        count_17 = codes[17]
        # переменная для хранения количества документов без замечаний
        without_mistakes = codes[0]
        for c in codes.keys():
            if c not in (0, 17):
                total_without_17 += codes[c]
        # переменная для хранения общего числа замечаний
        total_count = total_without_17 + count_17
        # переменная для хранения количества документов с повторно выданными замечаниями
        docs_with_repeated_codes = np.sum(data['Есть повторно выданные замечания'].loc[data['Отдел, разработчик']==name])

        report.loc[name, 'Всего замечаний, шт'] = total_count
        report.loc[name, 'Всего замечаний без 17 кода, шт'] = total_without_17
        report.loc[name, 'Доля документов с замечаниями, %'] = np.round(((1 - (without_mistakes / report.loc[name, 'Проверено документов, шт'])) * 100), 1)
        report.loc[name, 'Количество замечаний на 100 листов А4, шт'] = np.round(((total_count / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)
        report.loc[name, 'Количество замечаний на 100 листов А4 без 17 кода, шт'] = np.round(((total_without_17 / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)
        report.loc[name, 'Доля документов с повторно выданными замечаниями, %'] = np.round((docs_with_repeated_codes / (report.loc[name, 'Проверено документов, шт'] - without_mistakes) * 100), 1)

    # создаем переменные для хранения сводных показателей по предприятию
    all_docs = np.sum(report['Проверено документов, шт'])
    all_pages = np.sum(report['Проверено листов А4, шт'])
    all_mistakes = np.sum(report['Всего замечаний, шт'])
    all_mistakes_without_17 = np.sum(report['Всего замечаний без 17 кода, шт'])
    all_docs_with_mistakes = np.sum(report['Проверено документов, шт'] * (report['Доля документов с замечаниями, %'] / 100))
    all_docs_with_repeated_codes = np.sum(data['Есть повторно выданные замечания'])

    # добавляем в датафрейм строку со сводными показателями
    report.loc['Общая статистика по предприятию'] = [all_docs, all_pages, all_mistakes, all_mistakes_without_17, np.round((all_docs_with_mistakes / all_docs)*100, 1), np.round((all_mistakes / all_pages)*100, 1), np.round((all_mistakes_without_17 / all_pages)*100, 1), np.round((all_docs_with_repeated_codes / all_docs_with_mistakes)*100, 1)]

    return report

In [260]:
def make_codes_report(data: pd.DataFrame) -> pd.DataFrame:
    Codes = namedtuple('Codes', ['department', 'codes'])
    counts_of_codes = [Codes(dp, count_codes(data, dp)[0]) for dp in data['Отдел, разработчик'].unique()]

    cols = list(data['Отдел, разработчик'].unique())
    cols.insert(0, 'Шифр')
    report = pd.DataFrame(columns=[cols])

    for nt in counts_of_codes:
        report[nt.department] = nt.codes.values()

    report['Шифр'] = report.index

    return report

In [192]:
def make_codes_by_person_report(data: pd.DataFrame) -> pd.DataFrame:
    Codes = namedtuple('Codes', ['person', 'codes'])
    counts_of_codes = [Codes(pers, count_codes_by_person(data, pers)) for pers in data['Разработчик'].unique()]

    cols = list(data['Разработчик'].unique())
    cols.insert(0, 'Шифр')
    report = pd.DataFrame(columns=[cols])

    for nt in counts_of_codes:
        report[nt.person] = nt.codes.values()

    report['Шифр'] = report.index
    report = report.T
    report = report.drop('Шифр', axis=0)

    report.insert(0, 'Отдел', [report.index.values[i][0].split('_')[1] for i in range(report.index.shape[0])])

    return report

In [261]:
def make_time_report(data: pd.DataFrame) -> pd.DataFrame:
    report = pd.DataFrame(data['Срок проверки'].value_counts())
    return report

In [262]:
def make_report_by_doctype(data: pd.DataFrame) -> pd.DataFrame:
    litera_otrabotka = data.loc[data['Проект, тема, литера/ примечания']=='отработка на лит.']
    litera_assignment = data.loc[data['Проект, тема, литера/ примечания']=='присвоение литеры']
    tec_condition_extension = data.loc[data['Проект, тема, литера/ примечания']=='продление ТУ']
    new_and_correction = data.loc[data['Проект, тема, литера/ примечания'].isna()]
    correction = new_and_correction.loc[new_and_correction['Номер извещения'].notna()]
    new_docs = new_and_correction.loc[new_and_correction['Номер извещения'].isna()]

    cols = ['Подвыборка', 'Количество документов, шт', 'Количество листов А4']
    report = pd.DataFrame(columns=[cols])

    report.loc[len(report.index)] = ['Отработка на литеру', len(litera_otrabotka['Количество форматов А4']), litera_otrabotka['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Присвоение литеры', len(litera_assignment['Количество форматов А4']), litera_assignment['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Продление ТУ', len(tec_condition_extension['Количество форматов А4']), tec_condition_extension['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['По извещениям', len(correction['Количество форматов А4']), correction['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Новая документация', len(new_docs['Количество форматов А4']), new_docs['Количество форматов А4'].sum()]

    return report

# 5 Деление датасета на части

In [263]:
# разделим датасет на части по кварталам и полугодиям
data_1q = data[(data['Дата получения документа'] > '2024-01-01') & (data['Дата получения документа'] < '2024-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2024-04-01') & (data['Дата получения документа'] < '2024-07-01')]
data_3q = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2024-10-01')]
data_4q = data[(data['Дата получения документа'] >= '2024-10-01') & (data['Дата получения документа'] < '2025-01-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2024-07-01')]
data_2_half_year = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2025-01-01')]
data_full = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2025-01-01')]
# data_list = [data_1q, data_2q, data_3q, data_4q, data_1_half_year, data_2_half_year, data_full]
data_list = [data_2q, data_1_half_year]

In [264]:
# создадим аттрибуты с именем датасета
data_1q.name = '1_квартал'
data_2q.name = '2_квартал'
data_3q.name = '3_квартал'
data_4q.name = '4_квартал'
data_1_half_year.name = '1_полугодие'
data_2_half_year.name = '2_полугодие'
data_full.name = 'Весь_год'

In [265]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \n3 квартал: {data_3q.shape}  \n4 квартал: {data_4q.shape} \n1 полугодие: {data_1_half_year.shape}  \n2 полугодие: {data_2_half_year.shape} \nВесь год: {data_full.shape}')

1 квартал: (660, 14) 
2 квартал: (1216, 14) 
3 квартал: (0, 14)  
4 квартал: (0, 14) 
1 полугодие: (1876, 14)  
2 полугодие: (0, 14) 
Весь год: (1876, 14)


# 6 Сохранение отчетов

In [273]:
for d in data_list:
    with zipfile.ZipFile(f'{d.name}.zip', 'w') as zf:
        with zf.open(f'{d.name}_основной_отчет.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_main_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_основной_по_ошибкам.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_codes_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_ошибки_по_разработчику.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_codes_by_person_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_по_срокам.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_time_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_по_виду_документов.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_report_by_doctype(d)
                df.to_excel(writer)


<ipython-input-272-8868a6b7a81d>:36: RuntimeWarning: invalid value encountered in scalar divide
  report.loc[name, 'Доля документов с повторно выданными замечаниями, %'] = np.round((docs_with_repeated_codes / (report.loc[name, 'Проверено документов, шт'] - without_mistakes) * 100), 1)
<ipython-input-192-bd3e9eababbd>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)
<ipython-input-192-bd3e9eababbd>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)
